In [219]:
import numpy as np
from sklearn.cluster import KMeans
import cv2
from matplotlib import pyplot as plt
from scipy.cluster.vq import vq
import webbrowser

# Functions

In [220]:
def create_img_kp(key_p, img):
    const_size = 128
    x = int(key_p.pt[0])
    y = int(key_p.pt[1])
    img1_new = img[x:const_size+x, y:const_size+y]
    list_of_img_kp.append(img1_new)
    
    
def ClusterIndicesNumpy(clustNum, labels_array): #numpy 
    return np.where(labels_array == clustNum)[0]


def ClusterIndicesComp(clustNum, labels_array): #list comprehension
    return np.array([i for i, x in enumerate(labels_array) if x == clustNum])


def compute_detect_kp_descr(img_name):
    img = cv2.imread(img_name, 0)
    sift = cv2.xfeatures2d.SIFT_create()
    kp, des = sift.detectAndCompute(img, None)
    des_all.append(des)
    for i in range(len(kp)):
        create_img_kp(kp[i], img)
    
def write_img(name, ind):
    cv2.imwrite(name, ind)

# Find all descriptors

In [221]:
des_all = []
list_of_img_kp = []
compute_detect_kp_descr('1.jpg')
compute_detect_kp_descr('5.jpg')
descriptors_ = des_all[0][0]
for i in des_all:
    for descriptor in i:
        descriptors_ = np.vstack((descriptors_, descriptor))
        descriptors = np.float32(descriptors_)
descriptors = np.delete(descriptors_, 1, 0)

# Clusterisation

In [222]:
n_clusters = 50
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 10, 1.0)
ret,label,center=cv2.kmeans(descriptors,n_clusters,None,criteria,10,cv2.KMEANS_RANDOM_CENTERS)
idx,_ = vq(descriptors,center)

# Find maximal cluster size

In [223]:
arr_of_all = np.bincount(idx)
print(arr_of_all)
index_of_max = np.argmax(arr_of_all)
print(index_of_max)
list_ind_des = ClusterIndicesComp(index_of_max,label)

[ 1  8 18  1 23 15 56 11 19 43 14 59 57 67 22  2 26 30 38  1 45  4  2 11 20
 15  6 10  4 16  1  9 35 45 41  6  1  1 10 36 54  4 15 22 35 31  1 11  1 11]
13


# Report (HTML page)

In [225]:
#cv2.imshow('target', list_of_img_kp[list_ind_des[0]])
path_img = 'result/desc'
for i in range(1, 5):
    write_img(path_img + str(i) + '.jpg', list_of_img_kp[list_ind_des[i]])
webbrowser.open('file:///home/settnozz/PycharmProjects/Bow_img/result/start.html')


True